In [ ]:
# ---------- IMPORT MODULES ----------
from get_city_graph import get_city_graph
from get_city_data_from_db import get_city_data_from_db
from store_city_to_db import store_city_to_db
from get_or_create_run_config import get_or_create_run_config
from create_iteration import create_iteration
from generate_vehicles import generate_vehicles
from generate_vehicle_routes import generate_vehicle_routes
from generate_congestion import generate_congestion
from plot_congestion_heatmap import plot_congestion_heatmap, plot_congestion_heatmap_interactive
from filter_routes_for_qubo import filter_routes_for_qubo
from get_congestion_weights import get_congestion_weights
from normalize_congestion_weights import normalize_congestion_weights
from congestion_weights import congestion_weights
from qubo_matrix import qubo_matrix

# ---------- CONFIGURATION ----------
from sqlalchemy.orm import sessionmaker
from models import * #City, Node, Edge, RunConfig, Iteration, Vehicle, VehicleRoute, CongestionMap, RoutePoint  # adjust to your actual model imports


CITY_NAME = "Bratislava, Slovakia"#"Most pri Bratislave, Slovakia"
DIST_THRESH = 10
SPEED_DIFF_THRESH = 2
RUN_CONFIG_ID = 3#20      
ITERATION_ID = 4#1
API_KEY = ''

Session = sessionmaker(bind=engine)
session = Session()


city = session.query(City).filter_by(name=CITY_NAME).first()
_, edges = get_city_data_from_db(session, city.id)

In [2]:
def get_run_config_params(session, run_config_id, city_name):
    from sqlalchemy import text

    # Step 1: Get city_id
    city_id_result = session.execute(text("""
        SELECT id FROM cities WHERE name = :city_name
    """), {'city_name': city_name}).fetchone()

    if city_id_result is None:
        raise ValueError(f"No city found with name '{city_name}'")

    city_id = city_id_result[0]

    sql = text("""
        SELECT
            id,
            city_id,
            n_cars,
            k_alternatives,
            min_length,
            max_length,
            time_step,
            time_window,
            created_at
        FROM run_configs
        WHERE id = :run_config_id
    """)

    result = session.execute(sql, {'run_config_id': run_config_id, 'city_id': city_id}).fetchone()

    if result is None:
        raise ValueError(f"No run_config found with id={run_config_id}")

    # Return as dictionary
    keys = ['id', 'city_id', 'n_cars', 'k_alternatives', 'min_length',
            'max_length', 'time_step', 'time_window', 'created_at']
    return dict(zip(keys, result))


In [ ]:
config = get_run_config_params(session, run_config_id=RUN_CONFIG_ID, city_name = CITY_NAME)
print(config['n_cars'], config['k_alternatives'], config['min_length'])

In [4]:
def get_routes_df(session, run_config_id, iteration_id):
    from sqlalchemy import text
    import pandas as pd

    query = text("""
        SELECT
            vehicle_id,
            route_id,
            point_id,
            edge_id,
            lat,
            lon,
            time,
            speed,
            cardinal
        FROM trafficOptimization.route_points
        WHERE run_configs_id = :run_config_id
          AND iteration_id = :iteration_id
        ORDER BY vehicle_id, route_id, point_id
    """)

    result = session.execute(query, {
        'run_config_id': run_config_id,
        'iteration_id': iteration_id
    })

    return pd.DataFrame(result.fetchall(), columns=[
        'vehicle_id', 'route_id', 'point_id', 'edge_id',
        'lat', 'lon', 'time', 'speed', 'cardinal'
    ])


In [5]:
def get_congestion_df(session, run_config_id, iteration_id):
    from sqlalchemy import text
    import pandas as pd

    query = text("""
        SELECT
            edge_id,
            vehicle1,
            vehicle2,   
            vehicle1_route,
            vehicle2_route,
            congestion_score
        FROM trafficOptimization.congestion_map
        WHERE run_configs_id = :run_config_id
          AND iteration_id = :iteration_id
    """)

    result = session.execute(query, {
        'run_config_id': run_config_id,
        'iteration_id': iteration_id
    })

    return pd.DataFrame(result.fetchall(), columns=['edge_id', 'vehicle1', 'vehicle2', 'vehicle1_route', 'vehicle2_route', 'congestion_score'])


In [ ]:
session

In [ ]:
#routes_df = get_routes_df(session, RUN_CONFIG_ID, ITERATION_ID)
congestion_df = get_congestion_df(session, RUN_CONFIG_ID, ITERATION_ID)


# Step 8: Filter routes for QUBO
filtered_vehicles = filter_routes_for_qubo(congestion_df, threshold_percentile=0.9)
#print(filtered_vehicles)
N_FILTERED = len(filtered_vehicles)
print("Number of elements:", N_FILTERED)

# Step 9: Compute wights from congestion
weights_df = get_congestion_weights(session, RUN_CONFIG_ID, ITERATION_ID)
#print(weights_df)
weights_df.to_csv("files/weights_df.csv", index=False)

#weights_normalized = normalize_congestion_weights(weights_df, N_FILTERED, config['k_alternatives'], filtered_vehicles)
#weights_wo_normalization, max_weight = congestion_weights(weights_df, N_FILTERED, config['k_alternatives'], filtered_vehicles)
#print(weights_normalized)



In [ ]:
# Step 10: QUBO
Q, weights, w_c = qubo_matrix(N_FILTERED, config['k_alternatives'], weights_df, filtered_vehicles, lambda_strategy="normalized", fixed_lambda=1.0)

#for (q1, q2), value in Q.items():
#    print(f"Q[{q1}, {q2}] = {value:.4f}")


In [10]:
import numpy as np
import pandas as pd

def qubo_dict_to_dataframe(Q, size):
    matrix = np.zeros((size, size))
    for (i, j), v in Q.items():
        matrix[i][j] = v
        if i != j:
            matrix[j][i] = v  # ensure symmetry for display
    return pd.DataFrame(matrix), matrix

# Example usage
size = N_FILTERED * config['k_alternatives']
Q_df, matrix = qubo_dict_to_dataframe(Q, size)



In [11]:
Q_df.to_csv("files/qubo_matrix.csv", index=False)


In [12]:
from compute_shortest_routes import compute_shortest_routes

# Step : Plot heatmap
plot_map = plot_congestion_heatmap_interactive(edges, congestion_df,offset_deg=0.000025)
plot_map
plot_map.save("files/congestion_heatmap.html")

shortes_routes_dur_df = compute_shortest_routes(session, RUN_CONFIG_ID, ITERATION_ID, method="duration")
plot_map_dur = plot_congestion_heatmap_interactive(edges, shortes_routes_dur_df,offset_deg=0.000025)
plot_map_dur
plot_map_dur.save("files/shortest_routes_dur_congestion_heatmap.html")


shortes_routes_dis_df = compute_shortest_routes(session, RUN_CONFIG_ID, ITERATION_ID, method="distance")
plot_map_dis = plot_congestion_heatmap_interactive(edges, shortes_routes_dis_df,offset_deg=0.000025)
plot_map_dis
plot_map_dis.save("files/shortest_routes_dis_congestion_heatmap.html")

session.close()

In [ ]:
import pandas as pd
import numpy as np

# Adjust the path as needed
file_path = "files/qubo_matrix.csv"

# Read only the first N rows/columns for a quick check (e.g., N=100)
N = 100  # or set to a smaller/larger value as needed
df = pd.read_csv(file_path, nrows=N, usecols=range(N))

# Check off-diagonal values
negatives = []
for i in range(N):
    for j in range(N):
        if i != j and df.iloc[i, j] < 0:
            negatives.append((i, j, df.iloc[i, j]))

print(f"Found {len(negatives)} negative off-diagonal values in the first {N}x{N} block.")
if negatives:
    print("Sample negative off-diagonal values (row, col, value):")
    print(negatives[:10])
else:
    print("No negative off-diagonal values found in the first block.")

In [4]:
import time
import datetime
import numpy as np
import json
import gzip
import os
from pathlib import Path
from collections import defaultdict
from dimod import SimulatedAnnealingSampler, BinaryQuadraticModel
from dwave.system import EmbeddingComposite, DWaveSampler, LeapHybridSampler
from dwave.cloud import Client
from models import QAResult
import logging
from typing import Any, Dict

logger = logging.getLogger(__name__)

def get_api_token() -> str:
    """
    Retrieve the API token securely from environment variable or fallback.
    """
    return os.environ.get('QA_API_TOKEN', '123456456afsdgaeh')


def authenticate_with_token(token: str) -> bool:
    try:
        with Client(token=token) as client:
            # Log useful profile info
            logger.info(f"Connected as: {client.profile}")
            return True
    except Exception as e:
        logger.error(f"Failed to authenticate with D-Wave: {e}")
        return False



def qa_testing(
    Q: dict,
    run_config_id: int,
    iteration_id: int,
    session: Any,
    comp_type: str = 'hybrid',
    num_reads: int = 10
) -> Dict[str, Any]:
    """
    Run QUBO formulation for the car-to-trase assignment using a specified quantum/classical sampler.
    Requires API token for authentication (set QA_API_TOKEN env variable or fallback to default).

    Args:
        Q: QUBO matrix as a dictionary {(q1, q2): value}
        run_config_id: Run configuration ID
        iteration_id: Iteration number
        session: SQLAlchemy session
        comp_type: 'test', 'hybrid', or 'qpu'
        num_reads: Number of reads for classical or QPU runs

    Returns:
        dict: Results including assignment validity, assignment, energy, and duration.
    """
    # --- Authentication ---
    api_token = get_api_token()
    if authenticate_with_token(api_token):
        logger.info("Authentication successful. QA profile loaded.")
    else:
        logger.error("Authentication failed. Invalid API token.")
        raise PermissionError("Invalid API token for QA profile.")

    # --- QUBO to BQM ---
    bqm = BinaryQuadraticModel.from_qubo(Q)

    # --- Run sampler ---
    start_time = time.perf_counter()
    if comp_type == 'sa':
        sampler = SimulatedAnnealingSampler()
        response = sampler.sample(bqm, num_reads=num_reads, seed=42)
    elif comp_type == 'hybrid':
        sampler = LeapHybridSampler()
        response = sampler.sample(bqm)
    elif comp_type == 'qpu':
        sampler = EmbeddingComposite(DWaveSampler())
        response = sampler.sample(bqm, num_reads=num_reads)
    else:
        logger.error(f"Unknown comp_type: {comp_type}")
        raise ValueError(f"Unknown comp_type: {comp_type}")
    duration_qa = time.perf_counter() - start_time

    # --- Process results ---
    best_sample, energy = response.first

    # Infer n and t from Q size (placeholder logic)
    n = len(bqm.variables)
    t = 1

    # Check validity (placeholder: always True)
    assignment_valid = True
    assignment = list(best_sample.values())

    # --- Save QUBO matrix to file ---
    def save_qubo(Q, filepath):
        with gzip.open(filepath, 'wt', encoding='utf-8') as f:
            json.dump(Q, f)

    filename = f"run_{run_config_id}_iter_{iteration_id}.json.gz"
    qubo_dir = Path("qubo_matrices")
    filepath = qubo_dir / filename
    qubo_dir.mkdir(parents=True, exist_ok=True)
    save_qubo(Q, filepath)

    # --- Store results in DB ---
    result_record = QAResult(
        run_configs_id=run_config_id,
        iteration_id=iteration_id,
        lambda_strategy=None,
        lambda_value=None,
        comp_type=comp_type,
        num_reads=num_reads,
        n_vehicles=n,
        k_alternatives=t,
        weights=None,
        vehicle_ids=None,
        assignment_valid=int(assignment_valid),
        assignment=assignment,
        energy=energy,
        duration=duration_qa,
        qubo_path=str(filepath),
        created_at=datetime.datetime.utcnow()
    )
    session.add(result_record)
    session.commit()

    logger.info(f"QA testing complete: assignment_valid={assignment_valid}, energy={energy}, duration={duration_qa:.2f}s")

    return {
        'comp_type': comp_type,
        'num_reads': num_reads,
        'n_vehicles': n,
        'k_alternattives': t,
        'assignment_valid': assignment_valid,
        'assignment': assignment,
        'energy': energy,
        'qubo_path': str(filepath),
        'duration': duration_qa
    }


In [6]:
token = get_api_token()
print(token)
authenticate_with_token(token)



Failed to authenticate with D-Wave: 'Client' object has no attribute 'profile'


lSvr-9ba699944c113ae812bfd126c64ec90efb6d4b74


False

In [ ]:
import os
print(os.environ.get('QA_API_TOKEN'))